# Stage 3: Response Builder - Unit Tests

This notebook tests the **Response Builder** service in isolation.

**What it tests:**
- Agent execution (with mocks)
- Financial metrics extraction (mock LLM)
- Strategic analysis generation (mock LLM)
- Prepared content formatting
- Error handling for failed agents

In [ ]:
# Setup path
import sys
sys.path.insert(0, "../..")

from cmpt.services import (
    ResponseBuilderService,
    ContextBuilderOutput,
    ContentPrioritizationOutput,
    CompanyInfo,
    TemporalContext,
    PrioritizedSource,
    Subquery,
    DataSource,
    Priority,
)

## Helper: Create Mock Inputs

In [ ]:
def create_mock_context_output(
    company_name: str = "Apple Inc",
    ticker: str = "AAPL",
) -> ContextBuilderOutput:
    """Create mock ContextBuilderOutput."""
    return ContextBuilderOutput(
        company_info=CompanyInfo(
            name=company_name,
            ticker=ticker,
            industry="Technology",
            sector="Information Technology",
            market_cap="Large Cap",
        ),
        company_name=company_name,
        ticker=ticker,
        temporal_context=TemporalContext(
            meeting_date="2025-02-15",
            fiscal_quarter="1",
            fiscal_year="2025",
            news_lookback_days=30,
            filing_quarters=8,
        ),
    )


def create_mock_prioritization_output(
    ticker: str = "AAPL",
) -> ContentPrioritizationOutput:
    """Create mock ContentPrioritizationOutput."""
    return ContentPrioritizationOutput(
        prioritized_sources=[
            PrioritizedSource(source=DataSource.SEC_FILING, priority=Priority.PRIMARY, enabled=True),
            PrioritizedSource(source=DataSource.EARNINGS, priority=Priority.PRIMARY, enabled=True),
            PrioritizedSource(source=DataSource.NEWS, priority=Priority.SECONDARY, enabled=True),
        ],
        subqueries=[
            Subquery(agent="SEC_agent", query="Apple Inc", params={"ticker": ticker}),
            Subquery(agent="earnings_agent", query="Apple Inc", params={"ticker": ticker}),
            Subquery(agent="news_agent", query="Apple Inc", params={"ticker": ticker}),
        ],
        subqueries_by_agent={
            "SEC_agent": [Subquery(agent="SEC_agent", query="Apple Inc", params={"ticker": ticker})],
            "earnings_agent": [Subquery(agent="earnings_agent", query="Apple Inc", params={"ticker": ticker})],
            "news_agent": [Subquery(agent="news_agent", query="Apple Inc", params={"ticker": ticker})],
        },
        priority_distribution={"earnings_agent": 50, "news_agent": 30, "SEC_agent": 20},
    )

print("✓ Helper functions created")

## Test 1: Basic Execution (No LLM, No Agents)

In [ ]:
async def test_basic_execution():
    """Test basic service execution without LLM or agents."""
    # No LLM client, no agents - will use mock/fallback implementations
    service = ResponseBuilderService()
    
    context = create_mock_context_output()
    prioritization = create_mock_prioritization_output()
    
    output = await service.execute(context, prioritization)
    
    # Verify output structure
    assert output is not None
    assert output.company_name == "Apple Inc"
    
    print(f"✓ Company: {output.company_name}")
    print(f"✓ Agents Succeeded: {output.agents_succeeded}")
    print(f"✓ Agents Failed: {output.agents_failed}")
    print(f"✓ Has Financial Metrics: {output.financial_metrics is not None}")
    print(f"✓ Has Strategic Analysis: {output.strategic_analysis is not None}")
    print(f"✓ Has Prepared Content: {output.prepared_content is not None}")
    return output

output = await test_basic_execution()

## Test 2: Prepared Content Structure

In [ ]:
async def test_prepared_content():
    """Test that prepared content is properly formatted."""
    service = ResponseBuilderService()
    
    context = create_mock_context_output(company_name="Microsoft Corporation", ticker="MSFT")
    prioritization = create_mock_prioritization_output(ticker="MSFT")
    
    output = await service.execute(context, prioritization)
    
    # Verify prepared content
    assert output.prepared_content is not None
    
    # Should contain company name
    assert "Microsoft" in output.prepared_content
    
    print(f"✓ Prepared Content (first 500 chars):")
    print("-" * 50)
    print(output.prepared_content[:500])
    print("-" * 50)
    return output

output = await test_prepared_content()

## Test 3: Agent Results Structure

In [ ]:
async def test_agent_results():
    """Test that agent results are properly structured."""
    service = ResponseBuilderService()
    
    context = create_mock_context_output()
    prioritization = create_mock_prioritization_output()
    
    output = await service.execute(context, prioritization)
    
    print(f"✓ Agent Results:")
    for agent_name, result in output.agent_results.items():
        print(f"    {agent_name}:")
        print(f"      - Success: {result.get('success', 'N/A')}")
        print(f"      - Has Data: {result.get('data') is not None}")
        print(f"      - Error: {result.get('error', 'None')}")
    
    return output

output = await test_agent_results()

## Test 4: Financial Metrics Structure

In [ ]:
async def test_financial_metrics():
    """Test financial metrics structure (with mock data)."""
    service = ResponseBuilderService()
    
    context = create_mock_context_output()
    prioritization = create_mock_prioritization_output()
    
    output = await service.execute(context, prioritization)
    
    if output.financial_metrics:
        print(f"✓ Financial Metrics:")
        for key, value in list(output.financial_metrics.items())[:10]:
            if value is not None and not key.endswith('_citation'):
                print(f"    {key}: {value}")
    else:
        print("✓ No financial metrics (expected without LLM)")
    
    return output

output = await test_financial_metrics()

## Test 5: Strategic Analysis Structure

In [ ]:
async def test_strategic_analysis():
    """Test strategic analysis structure (with mock data)."""
    service = ResponseBuilderService()
    
    context = create_mock_context_output()
    prioritization = create_mock_prioritization_output()
    
    output = await service.execute(context, prioritization)
    
    if output.strategic_analysis:
        print(f"✓ Strategic Analysis:")
        for key, value in output.strategic_analysis.items():
            if isinstance(value, list):
                print(f"    {key}: {len(value)} items")
            elif value:
                print(f"    {key}: {str(value)[:50]}...")
    else:
        print("✓ No strategic analysis (expected without LLM)")
    
    return output

output = await test_strategic_analysis()

## Test 6: Timing Metrics

In [ ]:
async def test_timing_metrics():
    """Test that timing metrics are captured."""
    service = ResponseBuilderService()
    
    context = create_mock_context_output()
    prioritization = create_mock_prioritization_output()
    
    output = await service.execute(context, prioritization)
    
    assert output.timing_ms is not None
    
    print(f"✓ Timing Metrics:")
    for key, value in output.timing_ms.items():
        print(f"    {key}: {value:.2f}ms")
    
    # Should have total timing
    assert "total" in output.timing_ms or len(output.timing_ms) > 0
    return output

output = await test_timing_metrics()

## Test 7: Error Handling

In [ ]:
async def test_error_handling():
    """Test graceful handling of errors."""
    service = ResponseBuilderService()
    
    # Minimal context (missing some expected fields)
    context = ContextBuilderOutput(
        company_name="TestCorp",
    )
    prioritization = ContentPrioritizationOutput(
        prioritized_sources=[],
        subqueries=[],
        subqueries_by_agent={},
    )
    
    # Should not raise, should handle gracefully
    output = await service.execute(context, prioritization)
    
    assert output is not None
    print(f"✓ Service handled minimal input gracefully")
    print(f"✓ Company: {output.company_name}")
    print(f"✓ Errors: {output.errors}")
    return output

output = await test_error_handling()

## Test 8: Validation Results

In [ ]:
async def test_validation_results():
    """Test that validation results are included."""
    service = ResponseBuilderService()
    
    context = create_mock_context_output()
    prioritization = create_mock_prioritization_output()
    
    output = await service.execute(context, prioritization)
    
    if output.validation_results:
        print(f"✓ Validation Results:")
        if "validation_summary" in output.validation_results:
            summary = output.validation_results["validation_summary"]
            print(f"    Fields Checked: {summary.get('total_fields_checked', 0)}")
            print(f"    Fields with Values: {summary.get('fields_with_values', 0)}")
            print(f"    Sources Verified: {summary.get('sources_verified', 0)}")
    else:
        print("✓ No validation results (expected without financial metrics)")
    
    return output

output = await test_validation_results()

## Summary

In [ ]:
print("="*60)
print("  Stage 3: Response Builder - All Tests Passed!")
print("="*60)
print()
print("Tested:")
print("  ✓ Basic execution (no LLM, no agents)")
print("  ✓ Prepared content structure")
print("  ✓ Agent results structure")
print("  ✓ Financial metrics structure")
print("  ✓ Strategic analysis structure")
print("  ✓ Timing metrics")
print("  ✓ Error handling")
print("  ✓ Validation results")
print()
print("Note: Full LLM/agent tests require live services.")